<a href="https://colab.research.google.com/github/Kurt-Casteg/Proyecto_Almacenamiento/blob/main/Trabajo_Final_ACD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Trabajo Final Almacenamiento y Captura de Datos


<div>
<img src="https://i.ibb.co/v3CvVz9/udd-short.png" width="150"/>
    <br>
    <strong>Universidad del Desarrollo</strong><br>
    <em>Profesor: Carlos Pérez Pizarro </em><br>

</div>

*10 de enero de 2025*

**Nombre Estudiantes**:  
    - Kurt Castro   
    - Camilo Rivera   
    - Cristhian Solís  
    - Sofía Vits

In [ ]:
!apt-get update # Actualiza los paquetes
!apt install -y wget curl unzip # Instala herramientas necesarias
!apt install -y chromium-chromedriver # Instala Chromium y el WebDriver
!pip install selenium # Instala Selenium

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,197 kB]
Get:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,632 kB]
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 4,225 kB in 2s (2,810 kB/s)
Reading pack

## Carga de librerías

In [ ]:
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from selenium.webdriver.chrome.options import Options
import sqlite3

## Declaración de variables utilizadas

In [ ]:
# Variables iniciales
api_key = "insertar_token_de_google"
tipo_contrato = 'Arriendo' # venta, arriendo o arriendo_temporal
tipo_inmueble = 'Departamentos' # dpto, casa u oficina.
ubicacion_inmueble = 'las condes' # la comuna de la búsqueda
monto_minimo = 500000 # monto mínimo de la búsqueda
monto_maximo = 1300000 # monto máximo de la búsqueda
cant_paginas = 2 # número de páginas a recorrer
radio_busqueda = '300' # radio (en metros) de búsqueda de lugares cercanos
busqueda_rubros = ['restaurante', 'supermercado'] # rubro de lugares cercanos

In [ ]:
# Configuración de Chrome en modo headless
chrome_options = Options()
chrome_options.add_argument("--headless")  # Ejecuta en modo sin interfaz gráfica
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")



## Webscraping portalinmobiliario

In [ ]:
# Inicializar el navegador
driver = webdriver.Chrome(options=chrome_options)
driver.get('https://www.portalinmobiliario.com/')

In [ ]:
# Función para cerrar el banner de cookies
def cerrar_banner_cookies():
    try:
        # Verificar si el elemento existe y es visible en el DOM
        cookie_button = WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.ID, "newCookieDisclaimerButton"))
        )

        # Verificar si el botón es clickeable
        if cookie_button.is_displayed():
            cookie_button.click()
            print("Banner de cookies cerrado")
        else:
            print("El banner de cookies no está visible")
    except TimeoutException:
        print("No se encontró el banner de cookies")
    except Exception as e:
        print(f"Error inesperado al intentar cerrar el banner de cookies: {e}")

In [ ]:
try:
    cerrar_banner_cookies()
    wait = WebDriverWait(driver, 5)

    # Filtrar según botón de tipo contrato
    contrato_button = wait.until(EC.element_to_be_clickable((By.ID, ":R2l5r:-trigger")))
    contrato_button.click()
    print("Tipo de contrato")
    tipo_contrato_opcion = wait.until(EC.element_to_be_clickable((By.XPATH, f"//span[text()='{tipo_contrato}']")))
    tipo_contrato_opcion.click()
    print(f"Seleccionado: {tipo_contrato}")

    # Filtrar según botón de tipo inmueble
    inmueble_button = wait.until(EC.element_to_be_clickable((By.ID, ":R4l5r:-trigger")))
    inmueble_button.click()  # Hacer clic para abrir el dropdown de inmuebles
    print(f"Tipo de inmueble")
    tipo_inmueble_opcion = wait.until(EC.element_to_be_clickable((By.XPATH, f"//span[text()='{tipo_inmueble}']")))
    tipo_inmueble_opcion.click()
    print(f"Seleccionado: {tipo_inmueble}")

    # Ingresar comuna de búsqueda
    comuna_input = wait.until(EC.presence_of_element_located((By.ID, ":Rml5r:")))
    comuna_input.send_keys(ubicacion_inmueble)
    print(f"Comuna ingresada: {ubicacion_inmueble}")

    # Usar la primera recomendación
    primera_recomendacion = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[contains(@class, 'andes-list__item-action')]")))
    texto_recomendacion = primera_recomendacion.text
    print("Primera recomendación seleccionada")
    print(f"Texto de la primera recomendación: {texto_recomendacion}")
    primera_recomendacion.click()

    # Buscar
    buscar_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//span[text()='Buscar']")))
    buscar_button.click()
    print("Búsqueda realizada")
    time.sleep(2)

except Exception as e:
    print(f"Error ocurrió: {e}")

Banner de cookies cerrado
Tipo de contrato
Seleccionado: Arriendo
Tipo de inmueble
Seleccionado: Departamentos
Comuna ingresada: las condes
Primera recomendación seleccionada
Texto de la primera recomendación: Las Condes, RM (Metropolitana)
Búsqueda realizada


In [ ]:
try:
    cerrar_banner_cookies()
    wait = WebDriverWait(driver, 2)
    # Esperar y localizar el campo de "Monto mínimo" por su data-testid
    monto_minimo_input = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@data-testid="Minimum-price"]')))

    # Hacer clic para activar el campo, limpiarlo e ingresar el valor
    monto_minimo_input.click()
    monto_minimo_input.clear()  # Limpiar el campo antes de ingresar un valor
    monto_minimo_input.send_keys(str(monto_minimo))  # Ingresar monto mínimo
    print(f"Monto mínimo ingresado: {monto_minimo}")

    # Esperar y localizar el campo de "Monto mínimo" por su data-testid
    monto_maximo_input = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@data-testid="Maximum-price"]')))

    # Hacer clic para activar el campo, limpiarlo e ingresar el valor
    monto_maximo_input.click()
    monto_maximo_input.clear()  # Limpiar el campo antes de ingresar un valor
    monto_maximo_input.send_keys(str(monto_maximo))  # Ingresar monto mínimo
    print(f"Monto maximo ingresado: {monto_maximo}")

    # Esperar y localizar el botón de "Aplicar" por su data-testid
    aplicar_button = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@data-testid="submit-price"]')))
    # Hacer clic en el botón para aplicar el filtro
    aplicar_button.click()
    print("Filtro aplicado con éxito")

except Exception as e:
    print(f"Error ocurrió: {e}")

No se encontró el banner de cookies
Monto mínimo ingresado: 500000
Monto maximo ingresado: 1300000
Filtro aplicado con éxito


In [ ]:
def extraer_url():
    try:
        datos_departamentos = []  # Lista local para almacenar los datos
        pagina_actual = 1  # Iniciar contador de páginas

        while pagina_actual <= cant_paginas:
            print(f"Extrayendo datos de la página {pagina_actual}...")

            # Extraer los enlaces de los departamentos
            departamentos = wait.until(
                EC.presence_of_all_elements_located(
                    (By.XPATH, "//div[@class='poly-card__content']//h2[@class='poly-box poly-component__title']//a")
                )
            )
            for departamento in departamentos:
                enlace = departamento.get_attribute("href")
                datos_departamentos.append(enlace)  # Añadir el enlace a la lista

            # Navegación pagina siguiente
            try:
                siguiente_pagina = wait.until(
                    EC.element_to_be_clickable(
                        (By.XPATH, "//a[contains(@class, 'andes-pagination__link') and span[contains(text(), 'Siguiente')]]")
                    )
                )
                siguiente_pagina.click()  # Hacer clic en el botón "Siguiente"
                time.sleep(5)  # Esperar que cargue la siguiente página
                pagina_actual += 1  # Incrementar el contador de páginas
            except TimeoutException:
                print("Error: No se encontró el botón de siguiente página o la página está completamente cargada.")
                break  # Terminar si no hay más páginas
            except NoSuchElementException:
                print("Error: No se encontró el botón de siguiente página.")
                break
            except Exception as e:
                print(f"Error: {e}")
                break  # Terminar si ocurrió otro error

        return datos_departamentos

    except TimeoutException:
        print("Error: No se encontraron los elementos dentro del tiempo de espera.")
        return []
    except Exception as e:
        print(f"Error al extraer información: {e}")
        return []

# Llamar la función para extraer las URLs
datos_departamentos = extraer_url()
print("Datos extraídos:", datos_departamentos)

Extrayendo datos de la página 1...
Extrayendo datos de la página 2...
Datos extraídos: ['https://portalinmobiliario.com/MLC-1555797337-spot-nueva-kennedy-_JM#polycard_client=search-nordic&position=1&search_layout=grid&type=item&tracking_id=92f4875c-5223-4b78-bc67-7e009fecaac5', 'https://portalinmobiliario.com/MLC-1525161537-vespucio-switch-_JM#polycard_client=search-nordic&position=2&search_layout=grid&type=item&tracking_id=92f4875c-5223-4b78-bc67-7e009fecaac5', 'https://portalinmobiliario.com/MLC-2648931672-augusto-leguia-_JM#polycard_client=search-nordic&position=3&search_layout=grid&type=item&tracking_id=92f4875c-5223-4b78-bc67-7e009fecaac5', 'https://portalinmobiliario.com/MLC-1408634227-somma-asturias-edificio-multifamily-_JM#polycard_client=search-nordic&position=4&search_layout=grid&type=item&tracking_id=92f4875c-5223-4b78-bc67-7e009fecaac5', 'https://portalinmobiliario.com/MLC-2814352114-dpto-amoblado-remodelado-2d2b-los-militares-cerca-del-metro-_JM#polycard_client=search-nord

In [ ]:
# Crear un diccionario para almacenar las URLs, títulos, monedas y valores
dict_info = {}

# Iterar sobre las URLs en datos_departamentos
for href in datos_departamentos:
    try:
        driver.get(href)  # Navegar por cada URL
        wait.until(EC.presence_of_element_located((By.CLASS_NAME, "ui-pdp-title")))  # Esperar que cargue el título

        # Extraer el título
        titulo = driver.find_element(By.CLASS_NAME, "ui-pdp-title").text

        # Extraer la dirección
        try:
            location_element = driver.find_element(By.CSS_SELECTOR, "div.ui-vip-location__subtitle p")
            direccion = location_element.text
        except Exception as e:
            direccion = "Error al obtener dirección"
            print(f"Error al procesar dirección: {e}")

        # Extraer el precio
        try:
            contenedor_precio = driver.find_element(By.CLASS_NAME, "andes-money-amount")
            meta_precio = contenedor_precio.find_element(By.XPATH, ".//meta[@itemprop='price']")
            valor = meta_precio.get_attribute("content")
            simbolo_moneda = contenedor_precio.find_element(By.CLASS_NAME, "andes-money-amount__currency-symbol").text
            moneda = "UF" if simbolo_moneda == "UF" else "$"        # Determinar la moneda desde el elemento visible

        except Exception as e:
            moneda = "Error"
            valor = "Error"
            print(f"Error al procesar precio en {href}: {e}")

        # Guardar información en el diccionario
        dict_info[href] = {"titulo": titulo, "direccion": direccion, "moneda": moneda, "valor": valor}

    except Exception as e:

        print(f"Error al procesar {href}: {e}")
        dict_info[href] = {"titulo": "Error al obtener título", "direccion": "Error", "moneda": "Error", "valor": "Error"}

# Crear un DataFrame con los datos extraídos
df = pd.DataFrame.from_dict(dict_info, orient='index').reset_index()
df.columns = ['pag', 'titulo', 'direccion', 'moneda', 'valor']

# Mostrar el DataFrame
print("\nDatos extraídos desde portalinmobiliario")
df


Datos extraídos desde portalinmobiliario


,pag,titulo,direccion,moneda,valor
0,https://portalinmobiliario.com/MLC-1555797337-...,Spot Nueva Kennedy,"Av. Manquehue Nte. 958, Las Condes, Parque Ara...",UF,19.64
1,https://portalinmobiliario.com/MLC-1525161537-...,Vespucio Switch,"Av. Américo Vespucio Sur 345, Metro Escuela Mi...",$,974560
2,https://portalinmobiliario.com/MLC-2648931672-...,Augusto Leguía,"Augusto Leguía Nte. 70, Barrio El Golf, Las Co...",$,815000
3,https://portalinmobiliario.com/MLC-1408634227-...,Somma Asturias - Edificio Multifamily,"Asturias 77, Metro Escuela Militar, Las Condes...",$,790000
4,https://portalinmobiliario.com/MLC-2814352114-...,Dpto Amoblado Remodelado 2d+2b Los Militares C...,"Los Militares 4717, Metro Escuela Militar, Las...",UF,27
...,...,...,...,...,...
91,https://portalinmobiliario.com/MLC-1565590051-...,Arriendo Depto Las Malvas 1d1b1e1b - Metro Man...,"Las Malvas - Del Inca, Metro Manquehue - Apuma...",$,600000
92,https://portalinmobiliario.com/MLC-2805202374-...,Departamento Felipe Ii Bilbao Isabel La Católica,"Felipe Ii 4233, 7550509 Las Condes, Región Met...",$,950000
93,https://portalinmobiliario.com/MLC-1556452305-...,Lindo Depto A Pasos Del Metro,"Leonardo Da Vinci 6940, Las Condes, Metro Hern...",$,1300000
94,https://portalinmobiliario.com/MLC-2784615740-...,Lindo! Excelente Ubicación,"Las Condes, Barrio El Golf, Las Condes, RM (Me...",$,700000


In [ ]:
# Cerrar el navegador al finalizar
driver.quit()

## Webscraping del valor diario de la UF

In [ ]:
# Inicializar el navegador
driver2 = webdriver.Chrome(options=chrome_options)
url = "https://valoruf.cl/"
driver2.get(url)

try:
    # Buscar el elemento que contiene el valor de la UF usando su clase 'vpr'
    uf_element = driver2.find_element(By.CLASS_NAME, "vpr")
    valor_uf_texto = uf_element.text  # Extraer el texto del elemento
    print(f"El valor de la UF al día de hoy es: {valor_uf_texto}")

    # Convertir el valor al formato numérico
    valor_uf_numerico = float(valor_uf_texto.replace("$", "").replace(".", "").replace(",", ".").strip())

finally:
    # Cerrar el navegador
    driver2.quit()



El valor de la UF al día de hoy es: $ 38.426,60


In [ ]:
# convertir la columna valor a numerico
df['valor'] = pd.to_numeric(df['valor'], errors='coerce')

# Crear nueva columna con los valores en peso, si corresponde
df['valor_en_pesos'] = df.apply(
    lambda row: row['valor'] * valor_uf_numerico if row['moneda'] == 'UF' else row['valor'], axis=1
)

In [ ]:
df.head()

,pag,titulo,direccion,moneda,valor,valor_en_pesos
0,https://portalinmobiliario.com/MLC-1555797337-...,Spot Nueva Kennedy,"Av. Manquehue Nte. 958, Las Condes, Parque Ara...",UF,19.64,754698.424
1,https://portalinmobiliario.com/MLC-1525161537-...,Vespucio Switch,"Av. Américo Vespucio Sur 345, Metro Escuela Mi...",$,974560.00,974560.000
2,https://portalinmobiliario.com/MLC-2648931672-...,Augusto Leguía,"Augusto Leguía Nte. 70, Barrio El Golf, Las Co...",$,815000.00,815000.000
3,https://portalinmobiliario.com/MLC-1408634227-...,Somma Asturias - Edificio Multifamily,"Asturias 77, Metro Escuela Militar, Las Condes...",$,790000.00,790000.000
4,https://portalinmobiliario.com/MLC-2814352114-...,Dpto Amoblado Remodelado 2d+2b Los Militares C...,"Los Militares 4717, Metro Escuela Militar, Las...",UF,27.00,1037518.200


## Función para limpiar las direcciones

In [ ]:
import re

def process_address(address):
    # Expresión regular para detectar rangos como "1200 – 1800" o "1200-1800"
    match = re.search(r'(\d+)\s*[–\-]\s*(\d+)', address)
    if match:
        # Extraer los límites del rango
        start, end = map(int, match.groups())
        # Calcular el punto medio del rango
        midpoint = (start + end) // 2
        # Reemplazar el rango con el punto medio en la dirección
        address = re.sub(r'(\d+\s*[–\-]\s*\d+)', str(midpoint), address)
    return address

# Aplicar la función a la columna 'direccion'
df['direccion_ajustada'] = df['direccion'].apply(process_address)

## Creación base de datos

Luego de obtener los datos de la página portalinmobiliario.com mediante webscraping y los datos de latitud, longitud y lugares cercanos mediante el uso de las APIs Geocoding y Places de Google, se creará el esquema de la base de datos que almacenará la información obtenida.

In [ ]:
# Conexión a la base de datos SQLite (se crea si no existe)
conn = sqlite3.connect('portal_inmobiliario.db')

# Crear un cursor para ejecutar comandos SQL
cursor = conn.cursor()


# Definir el esquema de la base de datos
esquema_sql = """
BEGIN TRANSACTION;
DROP TABLE IF EXISTS "inmuebles";
CREATE TABLE IF NOT EXISTS "inmuebles" (
    "nombre_inmueble" TEXT,
    "tipo_inmueble" TEXT,
    "tipo_contrato" TEXT,
    "precio_pesos" INTEGER,
    "ubicacion_inmueble" TEXT,
    "direccion_inmueble" TEXT,
    "place_id" INTEGER,
    "latitud" REAL,
    "longitud" REAL,
    PRIMARY KEY("nombre_inmueble"),
    FOREIGN KEY("place_id") REFERENCES "coordenadas"
);
DROP TABLE IF EXISTS "lugares";
CREATE TABLE IF NOT EXISTS "lugares" (
    "nombre_lugar" TEXT,
    "tipo_lugar" TEXT,
    "ubicacion_lugar" TEXT,
    "direccion_lugar" TEXT,
    "place_id" INTEGER,
    "latitud" REAL,
    "longitud" REAL,
    "user_ratings_total" INTEGER,
    PRIMARY KEY("nombre_lugar"),
    FOREIGN KEY("place_id") REFERENCES "coordenadas"
);
DROP TABLE IF EXISTS "coordenadas";
CREATE TABLE IF NOT EXISTS "coordenadas" (
    "place_id" INTEGER,
    "latitud" REAL,
    "longitud" REAL,
    PRIMARY KEY("place_id")
);
COMMIT;
"""

# Ejecutar el esquema SQL
cursor.executescript(esquema_sql)


## Carga de los datos obtenidos

In [ ]:
# Función para cargar datos desde un archivo CSV a una tabla
def cargar_datos(ruta_archivo, tabla):
    df = pd.read_csv(ruta_archivo, sep=';')
    df.to_sql(tabla, conn, if_exists='append', index=False)

# Cargar datos en las tablas
cargar_datos('inmuebles.csv', 'inmuebles')
cargar_datos('lugares.csv', 'lugares')
cargar_datos('coordenadas.csv','coordenadas')

# Cerrar la conexión
conn.close()# Datos de ejemplo para cada tabla

## Consultas

Las consultas a realizar son:

¿Cuál es el valor promedio de los veinte arriendos de departamento más baratos de la comuna?

¿Cuál es la mediana de comentarios (user_ratings_total), de aquellos lugares cercanos, que tienen una valoración igual o superior a cuatro estrellas, y que corresponden a los 15 departamentos más baratos de la comuna?

In [ ]:
# Abrir conexión a la base de datos
conn = sqlite3.connect('portal_inmobiliario.db')

# Crear un objeto cursor para ejecutar comandos SQL para obtener el valor promedio de los veinte arriendos de departamento más baratos de la comuna
cursor = conn.cursor()

# Realizar la consulta para obtener
cursor.execute("SELECT precio_pesos  FROM inmuebles WHERE tipo_inmueble = "Departamento" AND tipo_contrato = "Arriendo" ")

# Recuperar y almacenar los resultados
valor_promedio_arriendos_baratos = cursor.fetchall()

# Retornar los resultados
for fila in valor_promedio_arriendos_baratos:
    print(fila)

# Cerrar la conexión
conn.close()

In [ ]:
# Abrir conexión a la base de datos
conn = sqlite3.connect('portal_inmobiliario.db')

# Crear un objeto cursor para ejecutar comandos SQL para obtener la mediana de comentarios (user_ratings_total),
# de aquellos lugares cercanos, que tienen una valoración igual o superior a cuatro estrellas, y que corresponden a los 15 departamentos más baratos de la comuna
cursor = conn.cursor()

# Realizar la consulta para obtener
cursor.execute("SELECT * FROM inmuebles")

# Recuperar y almacenar los resultados
valor_mediana_comentarios = cursor.fetchall()

# Retornar los resultados
for fila in valor_mediana_comentarios:
    print(fila)

# Cerrar la conexión
conn.close()